读取数据

In [17]:
import pandas as pd
import numpy as np
import pylab
import calendar
import numpy as np
import pandas as pd
import seaborn as sn
from scipy import stats
import missingno as msno
from datetime import datetime
import matplotlib.pyplot as plt
import warnings
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline


In [18]:
feature_columns=['holiday', 'workingday', 
       'season_1', 'season_2', 'season_3', 'season_4', 'weather_1',
       'weather_2', 'weather_3', 'weather_4', 'temp_scaled',
       'atemp_scaled', 'humidity_scaled', 'windspeed_scaled',
       'date_newyear_num', 'date_newyear_num_scaled', 'month_April',
       'month_August', 'month_December', 'month_February', 'month_January',
       'month_July', 'month_June', 'month_March', 'month_May',
       'month_November', 'month_October', 'month_September', 'hour_0',
       'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23',
       'weekday_Friday', 'weekday_Monday', 'weekday_Saturday',
       'weekday_Sunday', 'weekday_Thursday', 'weekday_Tuesday',
       'weekday_Wednesday', 'hour_workingday_0_0', 'hour_workingday_0_1',
       'hour_workingday_10_0', 'hour_workingday_10_1',
       'hour_workingday_11_0', 'hour_workingday_11_1',
       'hour_workingday_12_0', 'hour_workingday_12_1',
       'hour_workingday_13_0', 'hour_workingday_13_1',
       'hour_workingday_14_0', 'hour_workingday_14_1',
       'hour_workingday_15_0', 'hour_workingday_15_1',
       'hour_workingday_16_0', 'hour_workingday_16_1',
       'hour_workingday_17_0', 'hour_workingday_17_1',
       'hour_workingday_18_0', 'hour_workingday_18_1',
       'hour_workingday_19_0', 'hour_workingday_19_1',
       'hour_workingday_1_0', 'hour_workingday_1_1',
       'hour_workingday_20_0', 'hour_workingday_20_1',
       'hour_workingday_21_0', 'hour_workingday_21_1',
       'hour_workingday_22_0', 'hour_workingday_22_1',
       'hour_workingday_23_0', 'hour_workingday_23_1',
       'hour_workingday_2_0', 'hour_workingday_2_1', 'hour_workingday_3_0',
       'hour_workingday_3_1', 'hour_workingday_4_0', 'hour_workingday_4_1',
       'hour_workingday_5_0', 'hour_workingday_5_1', 'hour_workingday_6_0',
       'hour_workingday_6_1', 'hour_workingday_7_0', 'hour_workingday_7_1',
       'hour_workingday_8_0', 'hour_workingday_8_1', 'hour_workingday_9_0',
       'hour_workingday_9_1', 'hour_week_section_0', 'hour_week_section_1',
       'hour_week_section_2', 'hour_week_section_3', 'hour_week_section_4',
       'hour_week_section_5', 'hour_week_section_6', 'hour_week_section_7']



In [19]:
all_df=pd.read_csv("all_df20171217.csv")
X=all_df.loc[all_df.traintest=='train',feature_columns].values
y_casual=all_df.loc[all_df.traintest=='train'].casual.apply(lambda x: np.log1p(x)).values
y_regstered=all_df.loc[all_df.traintest=='train'].registered.apply(lambda x: np.log1p(x)).values
y_all=all_df.loc[all_df.traintest=='train','count'].apply(lambda x: np.log1p(x)).values

X_test=all_df.loc[all_df.traintest=='test',feature_columns].values
X_test_date=all_df.loc[all_df.traintest=='test','datetime'].values

随机森林来一把，太慢了。。。。。。。

In [31]:
def rmsle(y, y_):    
    log1 = np.nan_to_num(y)
    log2 = np.nan_to_num(y_)    
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from tqdm import *
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
parameters = {  
    "n_estimators": [2000],  
    #"max_depth":[5],
    #"min_samples_leaf": [1],  
    #"min_samples_split": [5],
    #"max_features":["sqrt"]
}
estimator = RandomForestRegressor(random_state=0, n_estimators=2000, n_jobs=-1)
gsearch = GridSearchCV(estimator, param_grid=parameters, scoring=rmsle_scorer, cv=5)
gsearch.fit(X=X,y=y_casual)
gsearch.grid_scores_, gsearch.best_params_, gsearch.best_score_

In [51]:
estimator = Ridge()
parameters = { 
    'alpha':[0.1, 1, 2, 3, 4, 10, 30,100,200]
}
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_ridge_m = GridSearchCV( estimator,
                          param_grid=parameters,
                          scoring = rmsle_scorer,
                          cv=5)

grid_ridge_m.fit(X=X,y=y_all)
grid_ridge_m.grid_scores_, grid_ridge_m.best_params_, grid_ridge_m.best_score_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.60267, std: 0.09695, params: {'alpha': 0.1},
  mean: -0.60242, std: 0.09671, params: {'alpha': 1},
  mean: -0.60221, std: 0.09654, params: {'alpha': 2},
  mean: -0.60201, std: 0.09641, params: {'alpha': 3},
  mean: -0.60183, std: 0.09630, params: {'alpha': 4},
  mean: -0.60085, std: 0.09578, params: {'alpha': 10},
  mean: -0.59932, std: 0.09423, params: {'alpha': 30},
  mean: -0.60570, std: 0.08907, params: {'alpha': 100},
  mean: -0.62485, std: 0.08358, params: {'alpha': 200}],
 {'alpha': 30},
 -0.59932024188446698)

In [52]:
pre=grid_ridge_m.predict(X)
print ("RMSLE Value For Ridge Regression: ",rmsle(y_all,pre))

RMSLE Value For Ridge Regression:  0.426928000074


In [69]:
estimator =Lasso()
parameters = { 
    'alpha':[0.1, 1, 2, 3, 4, 10, 30,100,200]
}
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)
grid_lasso_m = GridSearchCV( estimator,
                          param_grid=parameters,
                          scoring = rmsle_scorer,
                          cv=5)

grid_lasso_m.fit(X=X,y=y_all)
grid_lasso_m.grid_scores_, grid_lasso_m.best_params_, grid_lasso_m.best_score_

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


([mean: -0.97254, std: 0.10073, params: {'alpha': 0.1},
  mean: -1.41201, std: 0.06559, params: {'alpha': 1},
  mean: -1.41236, std: 0.06895, params: {'alpha': 2},
  mean: -1.41308, std: 0.07224, params: {'alpha': 3},
  mean: -1.41418, std: 0.07545, params: {'alpha': 4},
  mean: -1.42651, std: 0.08927, params: {'alpha': 10},
  mean: -1.43247, std: 0.08612, params: {'alpha': 30},
  mean: -1.43247, std: 0.08612, params: {'alpha': 100},
  mean: -1.43247, std: 0.08612, params: {'alpha': 200}],
 {'alpha': 0.1},
 -0.97254281807657661)

In [70]:
pre=grid_lasso_m.predict(X)
print ("RMSLE Value For Ridge Regression: ",rmsle(y_all,pre))

RMSLE Value For Ridge Regression:  0.922174293415


In [71]:
pd.DataFrame({"count":np.expm1(grid_lasso_m.predict(X_test)),"datetime":X_test_date}).to_csv("lasso.csv",index=False)

In [72]:
from sklearn.ensemble import GradientBoostingRegressor
gbm = GradientBoostingRegressor(n_estimators=4000,alpha=0.01); ### Test 0.41
gbm.fit(X=X,y=y_all)

NameError: name 'dataTrain' is not defined

In [89]:
preds = gbm.predict(X)
print ("RMSLE Value For Gradient Boost: ",rmsle(y_all,preds))

RMSLE Value For Gradient Boost:  0.197166611568


In [79]:
gbr=pd.DataFrame({"count":np.expm1(gbm.predict(X_test)),"datetime":X_test_date})

In [95]:
gbr.loc[gbr["count"]<=0,'count']=0

,count,datetime
0,24.704243,2011-01-20 00:00:00
1,7.017779,2011-01-20 01:00:00
2,4.154578,2011-01-20 02:00:00
3,3.102559,2011-01-20 03:00:00
4,1.870307,2011-01-20 04:00:00
